In [108]:
import os
import numpy as np
import tifffile as tiff
import imageio
from skimage import exposure

def pad_to_16(image):
    """Pad the image dimensions to be divisible by 16."""
    pad_h = (16 - image.shape[0] % 16) % 16
    pad_w = (16 - image.shape[1] % 16) % 16
    padded_image = np.pad(image, ((0, pad_h), (0, pad_w)), mode='constant')
    return padded_image

def process_and_save_webm(input_path, output_path):
    image_stack = tiff.imread(input_path)
    if image_stack.ndim==3:
        t, y, x = image_stack.shape
        c=1
    elif image_stack.ndim==4:
        t, c, y, x = image_stack.shape
    else:
        return

    # Convert to 8-bit and autoscale
    video_frames = []
    for t_idx, frame in enumerate(image_stack):  # Iterate over timepoints
        frame_output = []

        if c==1:
            channel = frame  # Shape: (y, x)
            channel = channel.astype(np.float32)
            # Compute percentiles for contrast adjustment
            lower_percentile = 1  # 1st percentile
            upper_percentile = 99  # 99th percentile

            lower_bound = np.percentile(channel, lower_percentile)
            upper_bound = np.percentile(channel, upper_percentile)

            # Scale channel to 8-bit using percentile-based contrast adjustment
            # channel -= lower_bound
            # channel /= (upper_bound - lower_bound) if (upper_bound - lower_bound) > 0 else 1
            np.clip(channel,lower_bound,upper_bound,out=channel)
            channel = exposure.rescale_intensity(channel, in_range=(np.min(channel), np.max(channel)), out_range=(0, 255))
            channel = channel.astype(np.uint8)
            channel = pad_to_16(channel)  # Pad to dimensions divisible by 16
            video_frames.append(channel)

        else:
            for c_idx in range(frame.shape[0]):  # Process each channel
                channel = frame[c_idx, :, :]  # Shape: (y, x)
                channel = channel.astype(np.float32)
                # Compute percentiles for contrast adjustment
                lower_percentile = 1  # 1st percentile
                upper_percentile = 99  # 99th percentile

                lower_bound = np.percentile(channel, lower_percentile)
                upper_bound = np.percentile(channel, upper_percentile)

                # Scale channel to 8-bit using percentile-based contrast adjustment
                # channel -= lower_bound
                # channel /= (upper_bound - lower_bound) if (upper_bound - lower_bound) > 0 else 1
                np.clip(channel,lower_bound,upper_bound,out=channel)
                channel = exposure.rescale_intensity(channel, in_range=(np.min(channel), np.max(channel)), out_range=(0, 255))
                channel = channel.astype(np.uint8)
                channel = pad_to_16(channel)  # Pad to dimensions divisible by 16
                frame_output.append(channel)

                # Combine channels into a single RGB-like frame (Green/Magenta mapping)
            if c == 2:  # Two channels
                combined_frame = np.stack((
                    frame_output[0],  # Magenta (R)
                    frame_output[1],  # Green (G)
                    frame_output[0]  # Magenta (R)
                ), axis=-1)
            else:
                combined_frame = np.stack(frame_output, axis=-1)  # Use all channels as-is
            video_frames.append(combined_frame)

    # Save the frames as a .webm video
    with imageio.get_writer(
        output_path,
        fps=12,
        format='FFMPEG',
        codec='libvpx-vp9',
        bitrate="10M",  # Set a high bitrate (e.g., 5 Mbps)
        quality=10,  # Higher quality scale (1=best, 10=worst)
        pixelformat="yuv420p",  # Ensures wide compatibility
    ) as writer:
        for frame in video_frames:
            writer.append_data(frame)


def find_and_process_tiffs(base_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, _, files in os.walk(base_directory):
        if 'MIPs' in root:  # Only process files in MIPs folders
            for file in files:
                if file.endswith('.tif'):
                    input_path = os.path.join(root, file)
                    relative_path = os.path.relpath(root, base_directory).replace(os.sep, "__")
                    #relative_path_flat = relative_path.replace("_", "_")
                    output_file = f"{relative_path}__{os.path.splitext(file)[0]}.webm"
                    output_path = os.path.join(output_directory, output_file)

                    print(f"Processing: {input_path} -> {output_path}")
                    process_and_save_webm(input_path, output_path)

if __name__ == "__main__":
    base_directory = input("Enter the base directory to search for MIPs folders: ").strip()
    output_directory = input("Enter the directory to save webm files: ").strip()
    find_and_process_tiffs(base_directory, output_directory)
    print("Processing complete!")


Processing: Z:\data\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM\CS1\Cell1\GPUdecon\MIPs\cell1_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM__CS1__Cell1__GPUdecon__MIPs__cell1_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM\CS1\cell3\GPUdecon\Cropped\MIPs\cell3_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM__CS1__cell3__GPUdecon__Cropped__MIPs__cell3_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM\CS1\cell3\GPUdecon\MIPs\cell3_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM__CS1__cell3__GPUdecon__MIPs__cell3_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-02-26-NDN-WIL2S-Phagocytosis_wtmNGmemFLM\CS1\cell4\GPUdecon\Cropped\MIPs\cell4_CamB_decon_comboMIP_z.tif -> X:\Phagoc

<tifffile.TiffFile 'cell4_CamB_decon_comboMIP_z.tif'> ImageJ series metadata invalid or corrupted file
<tifffile.TiffPages @8> invalid page offset 8408609
<tifffile.TiffFile 'cell5_CamB_decon_comboMIP_z.tif'> ImageJ series metadata invalid or corrupted file
<tifffile.TiffPages @8> invalid page offset 8408609


Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S\CS1\cell4\GPUdecon\MIPs\cell4_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S__CS1__cell4__GPUdecon__MIPs__cell4_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S\CS1\cell5\GPUdecon\MIPs\cell5_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S__CS1__cell5__GPUdecon__MIPs__cell5_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S\CS1\cell6\GPUdecon\MIPs\cell6_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S__CS1__cell6__GPUdecon__MIPs__cell6_CamB_decon_comboMIP_z.webm


<tifffile.TiffFile 'cell7_CamB_decon_comboMIP_z.tif'> ImageJ series metadata invalid or corrupted file
<tifffile.TiffPages @8> invalid page offset 8408610
<tifffile.TiffFile 'cell8_CamB_decon_comboMIP_z.tif'> ImageJ series metadata invalid or corrupted file
<tifffile.TiffPages @8> invalid page offset 8408609


Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S\CS1\cell7\GPUdecon\MIPs\cell7_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S__CS1__cell7__GPUdecon__MIPs__cell7_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S\CS1\cell8\GPUdecon\MIPs\cell8_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S__CS1__cell8__GPUdecon__MIPs__cell8_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S\CS1\cell9\GPUdecon\MIPs\cell9_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Green FLM-RTX-Calcein Violet-mscarlet WIL2S__CS1__cell9__GPUdecon__MIPs__cell9_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-07-02-NDN-Mneon Gree

...

<tifffile.TiffPages @8> invalid page offset 211947065


Processing: Z:\data\Phagocytosis-Bcell\2019-08-12-NDN-mScarletBCells-5uLRTX-6uL Calcein Violet-FLM\BMM Media 2\cell1\GPUdecon\MIPs\cell1_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-08-12-NDN-mScarletBCells-5uLRTX-6uL Calcein Violet-FLM__BMM Media 2__cell1__GPUdecon__MIPs__cell1_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-08-12-NDN-mScarletBCells-5uLRTX-6uL Calcein Violet-FLM\BMM Media 2\cell2\GPUdecon\MIPs\cell2_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-08-12-NDN-mScarletBCells-5uLRTX-6uL Calcein Violet-FLM__BMM Media 2__cell2__GPUdecon__MIPs__cell2_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-08-12-NDN-mScarletBCells-5uLRTX-6uL Calcein Violet-FLM\BMM Media 2\cell3\GPUdecon\MIPs\cell3_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-08-12-NDN-mScarletBCells-5uLRTX-6uL Calcein Violet-FLM__BMM Media 2__cell3__GPUdecon__MIPs__cell3_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-08

<tifffile.TiffFile 'cell6_CamB_decon_comboMIP_z.tif'> ImageJ series metadata invalid or corrupted file
<tifffile.TiffPages @8> invalid page offset 391374775


Processing: Z:\data\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL\BMM media FLM CS1\cell6-added bead\GPUdecon\MIPs\cell6_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL__BMM media FLM CS1__cell6-added bead__GPUdecon__MIPs__cell6_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL\BMM media FLM CS1\cell7\GPUdecon\MIPs\cell7_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL__BMM media FLM CS1__cell7__GPUdecon__MIPs__cell7_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL\BMM media FLM CS1\cell8\GPUdecon\MIPs\cell8_CamB_decon_comboMIP_z.tif -> X:\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL__BMM media FLM CS1__cell8__GPUdecon__MIPs__cell8_CamB_decon_comboMIP_z.webm
Processing: Z:\data\Phagocytosis-Bcell\2019-08-20-NDN-Phagocytosis 1uL RTX 5uL\BMM media FLM CS1\cell9\GPUde

TiffFileError: not a TIFF file b'RIFF'